# Assignment 3

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [1]:
# Load the libraries as required.
# Load environment variables
%load_ext dotenv
%dotenv ../05_src/.env

In [2]:
# Add src to path
import os
import sys
sys.path.append(os.getenv('SRC_DIR'))

# Standard libraries
import pandas as pd
import numpy as np

In [3]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


# Get X and Y

Create the features data frame and target data.

In [4]:
X = fires_dt.drop(columns='area')
Y = fires_dt['area']

# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [5]:
num_cols = X._get_numeric_data().columns.to_list()
cat_cols = X.drop(num_cols, axis=1).columns.to_list()


In [6]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

num_transforms1 = Pipeline([
    ('RobustScaler', RobustScaler())
])

num_transforms2 = Pipeline([
    ('PowerTransformer', PowerTransformer())
])

cat_transforms1 = Pipeline([
    ('OneHotEncoder', OneHotEncoder(handle_unknown='infrequent_if_exist', drop='if_binary'))
])

In [7]:
preproc1 = ColumnTransformer([
    ('num_transforms', num_transforms1, num_cols),
    ('cat_transforms', cat_transforms1, cat_cols)
])
preproc1

ColumnTransformer(transformers=[('num_transforms',
                                 Pipeline(steps=[('RobustScaler',
                                                  RobustScaler())]),
                                 ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                  'isi', 'temp', 'rh', 'wind', 'rain']),
                                ('cat_transforms',
                                 Pipeline(steps=[('OneHotEncoder',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='infrequent_if_exist'))]),
                                 ['month', 'day'])])

### Preproc 2

Create preproc2 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [8]:
num_cols_to_robust_transform = X[['ffmc', 'dc', 'isi', 'rain']].columns.tolist()
num_cols_to_power_transform = X[['coord_x', 'coord_y', 'dmc', 'temp', 'rh', 'wind']].columns.tolist()

In [9]:
preproc2 = ColumnTransformer([
    ('robust_transform', num_transforms1, num_cols_to_robust_transform),
    ('power_transform', num_transforms2, num_cols_to_power_transform),
    ('cat_transforms', cat_transforms1, cat_cols)
])
preproc2

ColumnTransformer(transformers=[('robust_transform',
                                 Pipeline(steps=[('RobustScaler',
                                                  RobustScaler())]),
                                 ['ffmc', 'dc', 'isi', 'rain']),
                                ('power_transform',
                                 Pipeline(steps=[('PowerTransformer',
                                                  PowerTransformer())]),
                                 ['coord_x', 'coord_y', 'dmc', 'temp', 'rh',
                                  'wind']),
                                ('cat_transforms',
                                 Pipeline(steps=[('OneHotEncoder',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='infrequent_if_exist'))]),
                                 ['month', 'day'])])

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [10]:
# Pipeline A = preproc1 + baseline
from sklearn.neighbors import KNeighborsRegressor

pipe_A = Pipeline([
    ('preprocessing', preproc1),
    ('baseline', KNeighborsRegressor())
])

pipe_A

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('RobustScaler',
                                                                   RobustScaler())]),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('OneHotEncoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='infrequent_if_exist'))]),
                                                  ['month', 'day'])])),
                ('baseline', KNeighborsRegressor())])

In [11]:
# Pipeline B = preproc2 + baseline
from sklearn.neighbors import KNeighborsRegressor

pipe_B = Pipeline([
    ('preprocessing', preproc2),
    ('baseline', KNeighborsRegressor())
])

pipe_B

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('robust_transform',
                                                  Pipeline(steps=[('RobustScaler',
                                                                   RobustScaler())]),
                                                  ['ffmc', 'dc', 'isi',
                                                   'rain']),
                                                 ('power_transform',
                                                  Pipeline(steps=[('PowerTransformer',
                                                                   PowerTransformer())]),
                                                  ['coord_x', 'coord_y', 'dmc',
                                                   'temp', 'rh', 'wind']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('OneHotEncoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='infrequent_if_exist'))]),
                                                  ['month', 'day'])])),
                ('baseline', KNeighborsRegressor())])

In [12]:
# Pipeline C = preproc1 + advanced model
from xgboost import XGBRegressor

pipe_C = Pipeline([
    ('preprocessing', preproc1),
    ('advanced', XGBRegressor())  # Initialize the XGBoost model here
])

pipe_C

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('RobustScaler',
                                                                   RobustScaler())]),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('OneHotEncoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='infrequent_if_exist'))]),
                                                  ['month', 'day'])])),
                (...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [13]:
# Pipeline D = preproc2 + advanced model
from sklearn.neighbors import KNeighborsRegressor

pipe_D = Pipeline([
    ('preprocessing', preproc2),
    ('advanced', XGBRegressor())
])

pipe_D
    

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('robust_transform',
                                                  Pipeline(steps=[('RobustScaler',
                                                                   RobustScaler())]),
                                                  ['ffmc', 'dc', 'isi',
                                                   'rain']),
                                                 ('power_transform',
                                                  Pipeline(steps=[('PowerTransformer',
                                                                   PowerTransformer())]),
                                                  ['coord_x', 'coord_y', 'dmc',
                                                   'temp', 'rh', 'wind']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('OneHotEncoder',
                                                                   OneHotEncod...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [14]:
pipe_A.get_params()

{'memory': None,
 'steps': [('preprocessing',
   ColumnTransformer(transformers=[('num_transforms',
                                    Pipeline(steps=[('RobustScaler',
                                                     RobustScaler())]),
                                    ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                     'isi', 'temp', 'rh', 'wind', 'rain']),
                                   ('cat_transforms',
                                    Pipeline(steps=[('OneHotEncoder',
                                                     OneHotEncoder(drop='if_binary',
                                                                   handle_unknown='infrequent_if_exist'))]),
                                    ['month', 'day'])])),
  ('baseline', KNeighborsRegressor())],
 'verbose': False,
 'preprocessing': ColumnTransformer(transformers=[('num_transforms',
                                  Pipeline(steps=[('RobustScaler',
                                

In [15]:
from sklearn.model_selection import GridSearchCV, train_test_split
scoring = ['neg_root_mean_squared_error', 'r2']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [16]:
param_grid_AB = {
    'baseline__n_neighbors': range(1, 21, 2),
    'baseline__weights': ['uniform', 'distance']
}


grid_cv_A = GridSearchCV(
    estimator = pipe_A,
    param_grid = param_grid_AB,
    scoring = scoring,
    refit = 'neg_root_mean_squared_error'
)
grid_cv_A.fit(X_train, Y_train)

/Users/viv/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/viv/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/viv/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/viv/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  war

GridSearchCV(estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num_transforms',
                                                                         Pipeline(steps=[('RobustScaler',
                                                                                          RobustScaler())]),
                                                                         ['coord_x',
                                                                          'coord_y',
                                                                          'ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('cat_transforms',
                                                                         Pipeline(steps=[('OneHotEncoder',
                                                                                          OneHotEncoder(drop='if_binary',
                                                                                                        handle_unknown='infrequent_if_exist'))]),
                                                                         ['month',
                                                                          'day'])])),
                                       ('baseline', KNeighborsRegressor())]),
             param_grid={'baseline__n_neighbors': range(1, 21, 2),
                         'baseline__weights': ['uniform', 'distance']},
             refit='neg_root_mean_squared_error',
             scoring=['neg_root_mean_squared_error', 'r2'])

In [17]:
# Access the cross-validation results using the property `.cv_results_`:
results_pipeA = grid_cv_A.cv_results_
results_pipeA = pd.DataFrame(results_pipeA)
results_pipeA.columns

results_pipeA = results_pipeA[['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'mean_test_neg_root_mean_squared_error',
       'std_test_neg_root_mean_squared_error',
       'rank_test_neg_root_mean_squared_error', 'mean_test_r2', 'std_test_r2', 'rank_test_r2']].assign(pipe = 'A')

best_res_pipeA = results_pipeA.loc[(results_pipeA['rank_test_neg_root_mean_squared_error'] == 1)]

best_res_pipeA

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,mean_test_neg_root_mean_squared_error,std_test_neg_root_mean_squared_error,rank_test_neg_root_mean_squared_error,mean_test_r2,std_test_r2,rank_test_r2,pipe
18,0.003247,0.000187,0.002275,0.000127,-39.621658,23.390407,1,-0.202696,0.312755,1,A


In [18]:
grid_cv_A.best_params_

{'baseline__n_neighbors': 19, 'baseline__weights': 'uniform'}

In [19]:
best_pipeA = grid_cv_A.best_estimator_

In [20]:
grid_cv_B = GridSearchCV(
    estimator = pipe_B,
    param_grid = param_grid_AB,
    scoring = scoring,
    refit = 'neg_root_mean_squared_error'
)
grid_cv_B.fit(X_train, Y_train)

grid_cv_B.best_params_

/Users/viv/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/viv/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/viv/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/viv/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  war

{'baseline__n_neighbors': 19, 'baseline__weights': 'uniform'}

In [21]:
# Access the cross-validation results using the property `.cv_results_`:
results_pipeB = grid_cv_B.cv_results_
results_pipeB = pd.DataFrame(results_pipeB)
results_pipeB.columns

results_pipeB = results_pipeA[['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'mean_test_neg_root_mean_squared_error',
       'std_test_neg_root_mean_squared_error',
       'rank_test_neg_root_mean_squared_error', 'mean_test_r2', 'std_test_r2', 'rank_test_r2']].assign(pipe = 'B')

best_res_pipeB = results_pipeB.loc[(results_pipeB['rank_test_neg_root_mean_squared_error'] == 1)]

best_res_pipeB

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,mean_test_neg_root_mean_squared_error,std_test_neg_root_mean_squared_error,rank_test_neg_root_mean_squared_error,mean_test_r2,std_test_r2,rank_test_r2,pipe
18,0.003247,0.000187,0.002275,0.000127,-39.621658,23.390407,1,-0.202696,0.312755,1,B


In [22]:
best_pipeB = grid_cv_B.best_estimator_

In [23]:
pipe_C.get_params()

{'memory': None,
 'steps': [('preprocessing',
   ColumnTransformer(transformers=[('num_transforms',
                                    Pipeline(steps=[('RobustScaler',
                                                     RobustScaler())]),
                                    ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                     'isi', 'temp', 'rh', 'wind', 'rain']),
                                   ('cat_transforms',
                                    Pipeline(steps=[('OneHotEncoder',
                                                     OneHotEncoder(drop='if_binary',
                                                                   handle_unknown='infrequent_if_exist'))]),
                                    ['month', 'day'])])),
  ('advanced',
   XGBRegressor(base_score=None, booster=None, callbacks=None,
                colsample_bylevel=None, colsample_bynode=None,
                colsample_bytree=None, device=None, early_stopping_rounds=None,
      

In [24]:
param_grid_CD = {
    'advanced__max_depth': range(1, 11, 1),
    'advanced__learning_rate': [0.1, 0.2, 0.3, 0.4, 0.5]
}

In [25]:
grid_cv_C = GridSearchCV(
    estimator = pipe_C,
    param_grid = param_grid_CD,
    scoring = scoring,
    refit = 'neg_root_mean_squared_error'
)
grid_cv_C.fit(X_train, Y_train)

grid_cv_C.best_params_

/Users/viv/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/viv/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/viv/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/viv/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  war

{'advanced__learning_rate': 0.1, 'advanced__max_depth': 1}

In [26]:
# Access the cross-validation results using the property `.cv_results_`:
results_pipeC = grid_cv_C.cv_results_
results_pipeC = pd.DataFrame(results_pipeC)
results_pipeC.columns

results_pipeC = results_pipeA[['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'mean_test_neg_root_mean_squared_error',
       'std_test_neg_root_mean_squared_error',
       'rank_test_neg_root_mean_squared_error', 'mean_test_r2', 'std_test_r2', 'rank_test_r2']].assign(pipe = 'C')

best_res_pipeC = results_pipeC.loc[(results_pipeC['rank_test_neg_root_mean_squared_error'] == 1)]

best_res_pipeC

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,mean_test_neg_root_mean_squared_error,std_test_neg_root_mean_squared_error,rank_test_neg_root_mean_squared_error,mean_test_r2,std_test_r2,rank_test_r2,pipe
18,0.003247,0.000187,0.002275,0.000127,-39.621658,23.390407,1,-0.202696,0.312755,1,C


In [27]:
best_pipeC = grid_cv_C.best_estimator_

In [28]:
grid_cv_D = GridSearchCV(
    estimator = pipe_D,
    param_grid = param_grid_CD,
    scoring = scoring,
    refit = 'neg_root_mean_squared_error'
)
grid_cv_D.fit(X_train, Y_train)

grid_cv_D.best_params_

/Users/viv/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/viv/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/viv/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/viv/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  war

{'advanced__learning_rate': 0.1, 'advanced__max_depth': 1}

In [29]:
# Access the cross-validation results using the property `.cv_results_`:
results_pipeD = grid_cv_D.cv_results_
results_pipeD = pd.DataFrame(results_pipeD)
results_pipeD.columns

results_pipeD = results_pipeA[['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'mean_test_neg_root_mean_squared_error',
       'std_test_neg_root_mean_squared_error',
       'rank_test_neg_root_mean_squared_error', 'mean_test_r2', 'std_test_r2', 'rank_test_r2']].assign(pipe = 'D')

best_res_pipeD = results_pipeD.loc[(results_pipeD['rank_test_neg_root_mean_squared_error'] == 1)]

best_res_pipeD

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,mean_test_neg_root_mean_squared_error,std_test_neg_root_mean_squared_error,rank_test_neg_root_mean_squared_error,mean_test_r2,std_test_r2,rank_test_r2,pipe
18,0.003247,0.000187,0.002275,0.000127,-39.621658,23.390407,1,-0.202696,0.312755,1,D


# Evaluate

+ Which model has the best performance?

In [31]:
compare_res_df = pd.concat([best_res_pipeA, best_res_pipeB, best_res_pipeC, best_res_pipeD])
compare_res_df 

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,mean_test_neg_root_mean_squared_error,std_test_neg_root_mean_squared_error,rank_test_neg_root_mean_squared_error,mean_test_r2,std_test_r2,rank_test_r2,pipe
18,0.003247,0.000187,0.002275,0.000127,-39.621658,23.390407,1,-0.202696,0.312755,1,A
18,0.003247,0.000187,0.002275,0.000127,-39.621658,23.390407,1,-0.202696,0.312755,1,B
18,0.003247,0.000187,0.002275,0.000127,-39.621658,23.390407,1,-0.202696,0.312755,1,C
18,0.003247,0.000187,0.002275,0.000127,-39.621658,23.390407,1,-0.202696,0.312755,1,D


# Export

+ Save the best performing model to a pickle file.

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

*(Answer here.)*

## Criteria

The [rubric](./assignment_3_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-3`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.